Test the most straightforward approach of user clustering for gesture classification:1. 
Assess user similarity (dendrogram? Distance in latent space (cosine and euclidean probably)?)1. 
Create datasets for every pair of users, and rank which pairs the most simila1. r
Use each dataset to train a classifi1. er
Goal thus is to show that more similar pairings produce higher performing mod1. els
May be able to scale this up to more than pairs (eg 3 users, 4 users, 1. etc)
Could be interesting to also do this but using the xlsx quasi-labels instead of pure data-driven approaches…


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import euclidean
from itertools import combinations

#from utils.subspace_clustering_helper_funcs import *
#from utils.ae_eval_and_vis import *
from utils.ae_torch_classes import *

In [3]:
# Choose a dataset and load in that data
## What to do about train/test split? ...

print("Loading")

data_path = "C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\saved_datasets\\filtered_datasets\\"
model_dir_path = "C:\\Users\\kdmen\\Desktop\\Research\\Repos\\fl-gestures\\models\\Embedding\Autoencoders\\"

full_emg_df = pd.read_pickle(data_path+'metadata_EMG_allgestures_allusers.pkl')

print(full_emg_df.shape)
full_emg_df.head()

Loading
(204800, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000004,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000005,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000004,0.000005,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.000002,0.000002,0.000002,0.000002,0.000005,0.000006,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.000003,0.000002,0.000002,0.000002,0.000005,0.000007,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


In [ ]:
# Do something about similarity? Would need to read the literature about this...

In [ ]:
# Iterate through pairs of users to create temp datasets
## For each temp dataset, use this to train and test models
### Probably LSTM, KNN, and SVC...

In [ ]:
# Embedding first..

# CREATE THE TRAINING SET
num_gestures = len(emg_training_users_df) // num_rows_per_gesture
num_features = emg_training_users_df.shape[1]
emg_train_loader = create_data_loader(emg_training_users_df, GestureDatasetAE, num_gestures, num_rows_per_gesture, num_features, batch_size, shuffle=True)
# CREATE THE TEST SET
num_features = emg_test_users_df.shape[1]
num_gestures = len(emg_test_users_df) // num_rows_per_gesture
emg_test_loader = create_data_loader(emg_test_users_df, GestureDatasetAE, num_gestures, num_rows_per_gesture, num_features, batch_size, shuffle=False)

hidden_dim_lst = [9]
input_dim = 16
RNNAE_9mir_EMGFull = RNNAutoencoder(input_dim, hidden_dim_lst, num_layers=None, seq_len=timesteps, progressive=False, mirror=True)
RNNAE_9mir_EMGFull.load_state_dict(torch.load(model_dir_path+'RNNAE_9mir_vallossp277_EMGFull.pth'))

average_test_loss, sample_data_lst, reconstructions_lst, latent_representations_lst = eval_on_testset_and_return_original_and_reconstructed(RNNAE_9mir_EMGFull, emg_test_loader)

latent_representations = np.concatenate(latent_representations_lst)
latent_representations_reshaped = latent_representations.reshape(latent_representations.shape[0], -1)


In [ ]:
latent_representations.shape

In [ ]:
latent_representations_reshaped.shape

In [4]:
full_emg_df['Participant'].unique()

array(['P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109',
       'P110', 'P111', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119',
       'P121', 'P122', 'P123', 'P124', 'P125', 'P126', 'P127', 'P128',
       'P131', 'P132', 'P004', 'P005', 'P006', 'P008', 'P010', 'P011'],
      dtype=object)

In [5]:
full_emg_df.columns

Index(['Participant', 'Gesture_ID', 'Gesture_Num', 'EMG1', 'EMG2', 'EMG3',
       'EMG4', 'EMG5', 'EMG6', 'EMG7', 'EMG8', 'EMG9', 'EMG10', 'EMG11',
       'EMG12', 'EMG13', 'EMG14', 'EMG15', 'EMG16'],
      dtype='object')

In [9]:
def calculate_gesture_distance(gesture1, gesture2):
    # Extract the EMG columns for distance calculation
    emg_columns = [col for col in gesture1.columns if col.startswith('EMG')]
    distance = euclidean(gesture1[emg_columns].values.flatten(), gesture2[emg_columns].values.flatten())
    return distance

In [10]:
df = full_emg_df

# Identify the unique participants
participants = df['Participant'].unique()

# Initialize a dictionary to store the distances
distances = {}

# Iterate through every pair of participants
for participant1, participant2 in combinations(participants, 2):
    print(f"Pair: ({participant1}, {participant2})")
    # Filter data for each participant
    data1 = df[df['Participant'] == participant1]
    data2 = df[df['Participant'] == participant2]
    
    # Assume both participants have the same number of gestures with the same Gesture_IDs
    gesture_ids = data1['Gesture_ID'].unique()
    
    participant_pair_distances = []
    
    for gesture_id in gesture_ids:
        # Get all gestures with the current Gesture_ID
        gestures1 = data1[data1['Gesture_ID'] == gesture_id]
        gestures2 = data2[data2['Gesture_ID'] == gesture_id]
        
        # Split the gestures into segments of 64 rows
        num_gestures = len(gestures1) // 64
        
        for i in range(num_gestures):
            gesture1 = gestures1.iloc[i*64:(i+1)*64]
            gesture2 = gestures2.iloc[i*64:(i+1)*64]
            
            # Calculate distance between the corresponding gestures
            distance = calculate_gesture_distance(gesture1, gesture2)
            participant_pair_distances.append(distance)
    
    # Store the average distance for the current pair of participants
    avg_distance = np.mean(participant_pair_distances)
    distances[(participant1, participant2)] = avg_distance

# Convert the distances dictionary to a dataframe for better readability
distances_df = pd.DataFrame(list(distances.items()), columns=['Participant_Pair', 'Average_Distance'])

# Print or save the distances dataframe
#print(distances_df)
# distances_df.to_csv('participant_distances.csv', index=False)  # Uncomment this line to save to a CSV file

print(distances_df.shape)
distances_df.head()

(496, 2)


,Participant_Pair,Average_Distance
0,"(P102, P103)",0.000476
1,"(P102, P104)",0.000436
2,"(P102, P105)",0.000826
3,"(P102, P106)",0.001047
4,"(P102, P107)",0.000439
